# Best Model Neural Network

In [1]:
import sys
sys.path.append("../src")

In [2]:
from features.features_dataset import FeaturesDataset
from models.fully_connected_classifier import FullyConnectedClassifier
from collections import Counter
from torch.utils.data import DataLoader

## Import Dataset

In [5]:
csv_path= r"..\data\processed\features_dataset.csv"
feat_dataset = FeaturesDataset.load_from_csv(csv_path, label_columns= ['id_measurement', 'id_performance', 'datetime', 'plant', 'generation', 'num_eurythmy', 'initial_second', 'eurythmy_letter'],
                                        variable_columns= ['mfcc_1_avg', 'mfcc_2_avg', 'mfcc_3_avg', 'mfcc_4_avg', 'mfcc_5_avg', 'mfcc_6_avg', 'mfcc_7_avg', 'mfcc_8_avg', 'mfcc_9_avg', 'mfcc_10_avg', 'mfcc_11_avg', 'mfcc_12_avg', 'mfcc_13_avg', 'mfcc_1_std', 'mfcc_2_std', 'mfcc_3_std', 'mfcc_4_std', 'mfcc_5_std', 'mfcc_6_std', 'mfcc_7_std', 'mfcc_8_std', 'mfcc_9_std', 'mfcc_10_std', 'mfcc_11_std', 'mfcc_12_std', 'mfcc_13_std', 'zero_crossing_rate', 'root_mean_square_energy', 'slope_sign_changes_ratio', 'hjorth_mobility', 'hjorth_complexity', 'mean', 'variance', 'standard_deviation', 'interquartile_range', 'skewness', 'kurtosis', 'dfa'],
                                        target_column=None)

In [6]:
feat_dataset.features.shape

(148682, 52)

In [ ]:
feat_dataset_path= r"..\data\processed\features_dataset"
feat_dataset = FeaturesDataset.load(file_path= feat_dataset_path)
feat_dataset.features.shape

## Prepare Data

In [7]:
feat_dataset.prepare_dataset(drop_constant= True, drop_flatness= True)

## RQ1

Is there any difference in the signals when someone is performing eurythmy?

In [ ]:
rq1= feat_dataset.return_subset_given_research_question(1)

In [ ]:
train_feat_dataset, val_feat_dataset, test_feat_dataset= rq1.split_dataset(split_by_wav= False)

In [ ]:
train_loader= DataLoader(train_feat_dataset, batch_size=32, shuffle=True)
val_loader= DataLoader(val_feat_dataset, batch_size=32, shuffle=True)

In [ ]:
klk

### Data preparation

In [ ]:
# Reduce the Dataset for the datapoints concerning RQ1

rq1_train_feat_dataset= train_feat_dataset.copy()
processed_train_indexes, train_targets= ped.get_indexes_and_targets_by_rq(1, rq1_train_feat_dataset.features)
rq1_train_feat_dataset.features= rq1_train_feat_dataset.features.iloc[processed_train_indexes]
rq1_train_feat_dataset.features.reset_index(drop=True, inplace=True)

rq1_test_feat_dataset= test_feat_dataset.copy()
processed_test_indexes, test_targets= ped.get_indexes_and_targets_by_rq(1, rq1_test_feat_dataset.features)
rq1_test_feat_dataset.features= rq1_test_feat_dataset.features.iloc[processed_test_indexes]
rq1_test_feat_dataset.features.reset_index(drop=True, inplace=True)

In [ ]:
# Reduce the features that are correlated in the training data
train_cols= rq1_train_feat_dataset.reduce_features(targets= train_targets, corr_threshold=0.75)
rq1_test_feat_dataset.keep_only_specified_variable_columns(train_cols)

In [ ]:
# Normalize features ?????
normalization_params= rq1_train_feat_dataset.normalize_features()
rq1_test_feat_dataset.apply_normalization(normalization_params)

In [ ]:
rq1_train_feat_dataset.objective_features.head()

In [ ]:
rq1_test_feat_dataset.objective_features.head()

In [ ]:
train_loader= rq1_train_feat_dataset.get_variable_features_loader(train_targets)
test_loader= rq1_test_feat_dataset.get_variable_features_loader(test_targets)

In [ ]:
def print_counts_and_percentages(values):
    count = Counter(values)
    total = sum(count.values())
    
    print("Counts and Percentages:")
    for key, value in count.items():
        percentage = (value / total) * 100
        print(f"Class {key}: Count = {value}, Percentage = {percentage:.2f}%")

In [ ]:
# Train dataset
print_counts_and_percentages(train_targets)

In [ ]:
# Test dataset
print_counts_and_percentages(test_targets)

### Training

In [ ]:
num_epochs = 50

In [ ]:
input_size= len(rq1_train_feat_dataset.variable_columns)
output_size= 2
num_epochs = 50

parameters = {
    'learning_rate': 0.001,
    'dense_units': 128,
    'dense_layers': 1,
    'dropout_rate': 0.2,
    'early_stopping_patience': 7
}

model = FullyConnectedClassifier(input_size= input_size, output_size= output_size, parameters= parameters)

In [ ]:
model.train_model(train_loader, test_loader, num_epochs)

### Evaluation

In [ ]:
test_predictions = model.predict(test_loader)

In [ ]:
actual_labels = []
for _, labels in test_loader:
    actual_labels.extend(labels.tolist())

In [ ]:
correct_predictions = sum(p == t for p, t in zip(test_predictions, actual_labels))
accuracy = correct_predictions / len(actual_labels)
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming your task is a classification task
precision = precision_score(actual_labels, test_predictions, average='macro')
recall = recall_score(actual_labels, test_predictions, average='macro')
f1 = f1_score(actual_labels, test_predictions, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(actual_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)

## RQ2

In [ ]:
### Data preparation

# Reduce the Dataset for the datapoints concerning rq2

rq2_train_feat_dataset= train_feat_dataset.copy()
processed_train_indexes, train_targets= ped.get_indexes_and_targets_by_rq(2, rq2_train_feat_dataset.features)
rq2_train_feat_dataset.features= rq2_train_feat_dataset.features.iloc[processed_train_indexes]
rq2_train_feat_dataset.features.reset_index(drop=True, inplace=True)

rq2_test_feat_dataset= test_feat_dataset.copy()
processed_test_indexes, test_targets= ped.get_indexes_and_targets_by_rq(2, rq2_test_feat_dataset.features)
rq2_test_feat_dataset.features= rq2_test_feat_dataset.features.iloc[processed_test_indexes]
rq2_test_feat_dataset.features.reset_index(drop=True, inplace=True)

# Reduce the features that are correlated in the training data
train_cols= rq2_train_feat_dataset.reduce_features(targets= train_targets, corr_threshold=0.75)
rq2_test_feat_dataset.keep_only_specified_variable_columns(train_cols)

# Normalize features
normalization_params= rq2_train_feat_dataset.normalize_features()
rq2_test_feat_dataset.apply_normalization(normalization_params)

rq2_train_feat_dataset.objective_features.head()

rq2_test_feat_dataset.objective_features.head()

train_loader= rq2_train_feat_dataset.get_variable_features_loader(train_targets)
test_loader= rq2_test_feat_dataset.get_variable_features_loader(test_targets)

# Train dataset
print_counts_and_percentages(train_targets)

# Test dataset
print_counts_and_percentages(test_targets)

### Training

num_epochs = 50

input_size= len(rq2_train_feat_dataset.variable_columns)
output_size= 3
num_epochs = 50

parameters = {
    'learning_rate': 0.0003,
    'dense_units': 128,
    'dense_layers': 2,
    'dropout_rate': 0.2
}

model = FullyConnectedClassifier(input_size= input_size, output_size= output_size, parameters= parameters)

model.train_model(train_loader, test_loader, num_epochs)

### Evaluation

test_predictions = model.predict(test_loader)

actual_labels = []
for _, labels in test_loader:
    actual_labels.extend(labels.tolist())

correct_predictions = sum(p == t for p, t in zip(test_predictions, actual_labels))
accuracy = correct_predictions / len(actual_labels)
print(f"Accuracy: {accuracy:.4f}")


from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming your task is a classification task
precision = precision_score(actual_labels, test_predictions, average='macro')
recall = recall_score(actual_labels, test_predictions, average='macro')
f1 = f1_score(actual_labels, test_predictions, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(actual_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)

## RQ3

In [ ]:
### Data preparation

# Reduce the Dataset for the datapoints concerning rq3

rq3_train_feat_dataset= train_feat_dataset.copy()
processed_train_indexes, train_targets= ped.get_indexes_and_targets_by_rq(3, rq3_train_feat_dataset.features)
rq3_train_feat_dataset.features= rq3_train_feat_dataset.features.iloc[processed_train_indexes]
rq3_train_feat_dataset.features.reset_index(drop=True, inplace=True)

rq3_test_feat_dataset= test_feat_dataset.copy()
processed_test_indexes, test_targets= ped.get_indexes_and_targets_by_rq(3, rq3_test_feat_dataset.features)
rq3_test_feat_dataset.features= rq3_test_feat_dataset.features.iloc[processed_test_indexes]
rq3_test_feat_dataset.features.reset_index(drop=True, inplace=True)

# Reduce the features that are correlated in the training data
train_cols= rq3_train_feat_dataset.reduce_features(targets= train_targets, corr_threshold=0.75)
rq3_test_feat_dataset.keep_only_specified_variable_columns(train_cols)

# Normalize features
normalization_params= rq3_train_feat_dataset.normalize_features()
rq3_test_feat_dataset.apply_normalization(normalization_params)

rq3_train_feat_dataset.objective_features.head()

rq3_test_feat_dataset.objective_features.head()

train_loader= rq3_train_feat_dataset.get_variable_features_loader(train_targets)
test_loader= rq3_test_feat_dataset.get_variable_features_loader(test_targets)

# Train dataset
print_counts_and_percentages(train_targets)

# Test dataset
print_counts_and_percentages(test_targets)

### Training

num_epochs = 50

input_size= len(rq3_train_feat_dataset.variable_columns)
output_size= 4
num_epochs = 50

parameters = {
    'learning_rate': 0.0003,
    'dense_units': 64,
    'dense_layers': 1,
    'dropout_rate': 0
}

model = FullyConnectedClassifier(input_size= input_size, output_size= output_size, parameters= parameters)

model.train_model(train_loader, test_loader, num_epochs)

### Evaluation

test_predictions = model.predict(test_loader)

actual_labels = []
for _, labels in test_loader:
    actual_labels.extend(labels.tolist())

correct_predictions = sum(p == t for p, t in zip(test_predictions, actual_labels))
accuracy = correct_predictions / len(actual_labels)
print(f"Accuracy: {accuracy:.4f}")


from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming your task is a classification task
precision = precision_score(actual_labels, test_predictions, average='macro')
recall = recall_score(actual_labels, test_predictions, average='macro')
f1 = f1_score(actual_labels, test_predictions, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(actual_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)

## RQ4

In [ ]:
### Data preparation

# Reduce the Dataset for the datapoints concerning rq4

rq4_train_feat_dataset= train_feat_dataset.copy()
processed_train_indexes, train_targets= ped.get_indexes_and_targets_by_rq(4, rq4_train_feat_dataset.features)
rq4_train_feat_dataset.features= rq4_train_feat_dataset.features.iloc[processed_train_indexes]
rq4_train_feat_dataset.features.reset_index(drop=True, inplace=True)

rq4_test_feat_dataset= test_feat_dataset.copy()
processed_test_indexes, test_targets= ped.get_indexes_and_targets_by_rq(4, rq4_test_feat_dataset.features)
rq4_test_feat_dataset.features= rq4_test_feat_dataset.features.iloc[processed_test_indexes]
rq4_test_feat_dataset.features.reset_index(drop=True, inplace=True)

# Reduce the features that are correlated in the training data
train_cols= rq4_train_feat_dataset.reduce_features(targets= train_targets, corr_threshold=0.75)
rq4_test_feat_dataset.keep_only_specified_variable_columns(train_cols)

# Normalize features
normalization_params= rq4_train_feat_dataset.normalize_features()
rq4_test_feat_dataset.apply_normalization(normalization_params)

rq4_train_feat_dataset.objective_features.head()

rq4_test_feat_dataset.objective_features.head()

train_loader= rq4_train_feat_dataset.get_variable_features_loader(train_targets)
test_loader= rq4_test_feat_dataset.get_variable_features_loader(test_targets)

# Train dataset
print_counts_and_percentages(train_targets)

# Test dataset
print_counts_and_percentages(test_targets)

### Training

num_epochs = 50

input_size= len(rq4_train_feat_dataset.variable_columns)
output_size= 4
num_epochs = 50

parameters = {
    'learning_rate': 0.0003,
    'dense_units': 64,
    'dense_layers': 2,
    'dropout_rate': 0.2
}

model = FullyConnectedClassifier(input_size= input_size, output_size= output_size, parameters= parameters)

model.train_model(train_loader, test_loader, num_epochs)

### Evaluation

test_predictions = model.predict(test_loader)

actual_labels = []
for _, labels in test_loader:
    actual_labels.extend(labels.tolist())

correct_predictions = sum(p == t for p, t in zip(test_predictions, actual_labels))
accuracy = correct_predictions / len(actual_labels)
print(f"Accuracy: {accuracy:.4f}")


from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming your task is a classification task
precision = precision_score(actual_labels, test_predictions, average='macro')
recall = recall_score(actual_labels, test_predictions, average='macro')
f1 = f1_score(actual_labels, test_predictions, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(actual_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)